# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
!pip install selenium

In [2]:
from selenium import webdriver # controlar o navegador
from selenium.webdriver.common.keys import Keys # controlar as teclas
from selenium.webdriver.common.by import By # localizar os itens no navegador

navegador = webdriver.Chrome()

navegador.get("https://google.com")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do Dólar")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').click()

cotacao_dol = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

navegador.get("https://google.com")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação do Euro")

navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]').click()

cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")

cotacao_ouro = cotacao_ouro.replace(",", ".")

print(cotacao_euro)

print(cotacao_dol)

print(cotacao_ouro)

navegador.quit()

5.538855000000001
5.075
325.06


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [3]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

display(tabela)

Produtos  Preço Original  Moeda  Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar        5          4999.95    1.40   
1   Carro Renault         4500.00   Euro        6         27000.00    2.00   
2   Notebook Dell          899.99  Dólar        5          4499.95    1.70   
3          IPhone          799.00  Dólar        5          3995.00    1.70   
4      Carro Fiat         3000.00   Euro        6         18000.00    1.90   
5  Celular Xiaomi          480.48  Dólar        5          2402.40    2.00   
6        Joia 20g           20.00   Ouro      350          7000.00    1.15   

   Preço de Venda  
0        6999.930  
1       54000.000  
2        7649.915  
3        6791.500  
4       34200.000  
5        4804.800  
6        8050.000

- Atualizando os preços e o cálculo do Preço Final

In [4]:
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dol)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

tabela["Preçode de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

Produtos  Preço Original  Moeda     Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar    5.075000       5074.94925    1.40   
1   Carro Renault         4500.00   Euro    5.538855      24924.84750    2.00   
2   Notebook Dell          899.99  Dólar    5.075000       4567.44925    1.70   
3          IPhone          799.00  Dólar    5.075000       4054.92500    1.70   
4      Carro Fiat         3000.00   Euro    5.538855      16616.56500    1.90   
5  Celular Xiaomi          480.48  Dólar    5.075000       2438.43600    2.00   
6        Joia 20g           20.00   Ouro  325.060000       6501.20000    1.15   

   Preço de Venda  Preçode de Venda  
0        6999.930       7104.928950  
1       54000.000      49849.695000  
2        7649.915       7764.663725  
3        6791.500       6893.372500  
4       34200.000      31571.473500  
5        4804.800       4876.872000  
6        8050.000       7476.380000

### Agora vamos exportar a nova base de preços atualizada

In [5]:
tabela.to_excel("ProdutosNovo.xlsx", index=False)